In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [2]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['SHGData1_3W_07042017.txt', 'SHGData1_3Watt_07042017.txt', 'SHGData1_4Watt_07042017.txt', 'SHGData1_2Watt_07042017.txt', 'SHGData1_1Watt_07042017.txt', 'SHGData1_0Watt_07042017.txt', 'SHGData0_9_2Watt_07042017.txt', 'SHGData0_8_2Watt_07042017.txt', 'SHGData0_7_2Watt_07042017.txt', 'SHGData0_6_2Watt_07042017.txt', 'SHGData0_5_2Watt_07042017.txt', 'SHGData0_4_2Watt_07042017.txt', 'SHGData0_3_2Watt_07042017.txt', 'SHGData0_2_2Watt_07042017.txt', 'SHGData0_1_2Watt_07042017.txt', 'SHGData0_0_2Watt_07042017.txt']


In [3]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [4]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [6]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (499*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay = []
    for i in range(0,len(x)):
        timedelay = ((x[i]*itr + offset)*25.4*2/(10**-12*c))-T1
        timeDelay.append(timedelay)
    return(timeDelay)

In [57]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,300):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(300,len(allFiles))
PDvalues = PDvalues.values.reshape(300,len(allFiles)) 
timeDelay = ConvertToTimeDelay(times,itr,offset) # function converts to timedelay

MeanPD = PDvalues.mean(axis = 0) #Getting mean of all the data
StDevPD = PDvalues.std(axis = 0) # Getting Standard Deviation of the data
ErrOfMean = (np.delete(StDevPD,0))/(300**0.5) # Standard error of mean stdev/sqrt(#ofdatapoints)
Background = MeanPD[-1] # Background is the last element in the allFiles
ActualPD = np.absolute(MeanPD - Background)
ActualPD2 = np.delete(ActualPD,0)
LogPD = np.log10(np.delete(ActualPD,0))
LaserPower = np.array([110,120,100,91,81,72,64,58,48,40,30,22,14,6])
LogLaser = np.log10(LaserPower)

fit = np.polyfit(LogLaser,LogPD[:-1],1,w=ErrOfMean[:-1])
f = np.poly1d(fit)
pPower = np.linspace(LogLaser[1],LogLaser[-1],100)
f_new = f(pPower)
print(f_new)
print(f)

plt.plot(LaserPower,ActualPD2[:-1],'k*')
#plt.plot(pPower,f_new,'b-')
plt.show()

[-0.71916886 -0.73366843 -0.74816799 -0.76266756 -0.77716712 -0.79166669
 -0.80616625 -0.82066582 -0.83516538 -0.84966495 -0.86416451 -0.87866408
 -0.89316364 -0.90766321 -0.92216277 -0.93666233 -0.9511619  -0.96566146
 -0.98016103 -0.99466059 -1.00916016 -1.02365972 -1.03815929 -1.05265885
 -1.06715842 -1.08165798 -1.09615755 -1.11065711 -1.12515668 -1.13965624
 -1.15415581 -1.16865537 -1.18315493 -1.1976545  -1.21215406 -1.22665363
 -1.24115319 -1.25565276 -1.27015232 -1.28465189 -1.29915145 -1.31365102
 -1.32815058 -1.34265015 -1.35714971 -1.37164928 -1.38614884 -1.4006484
 -1.41514797 -1.42964753 -1.4441471  -1.45864666 -1.47314623 -1.48764579
 -1.50214536 -1.51664492 -1.53114449 -1.54564405 -1.56014362 -1.57464318
 -1.58914275 -1.60364231 -1.61814188 -1.63264144 -1.647141   -1.66164057
 -1.67614013 -1.6906397  -1.70513926 -1.71963883 -1.73413839 -1.74863796
 -1.76313752 -1.77763709 -1.79213665 -1.80663622 -1.82113578 -1.83563535
 -1.85013491 -1.86463447 -1.87913404 -1.8936336  -1.

In [39]:
LogPD[:-1]

array([-0.74824825, -0.68679456, -0.7770582 , -0.86607274, -0.89718797,
       -0.94841404, -1.01769409, -1.13194952, -1.16813449, -1.28008894,
       -1.42324414, -1.50699298, -1.73069725, -2.08074753])

In [41]:
len(LogLaser)

14

In [55]:
len(ActualPD[:-1])

15

In [43]:
len(ErrOfMean[:-1])

15